In [2]:
from sagemaker.workflow.function_step import step
from sagemaker.workflow.pipeline import Pipeline
import sagemaker
from sagemaker.workflow.parameters import ParameterInteger

# FUNCIONES

In [57]:
pipeline_name = "pipeline-prueba"
role = sagemaker.get_execution_role()
#instance_type = "ml.m5.xlarge"
#instance_type = "ml.t3.medium"
instance_type = "ml.m5.large"
periodo = ParameterInteger(name="PeriodoCarga", default_value=202503)

In [58]:
@step(
    name="DataPull",
    instance_type=instance_type,
)
def data_pull(data_source:str, periodo:int)->tuple[str,str]:
    print(f"Data pull from {data_source}")
    print(f"Periodo:{periodo}")
    return "s3://train_path", "s3://test_path"
    

In [59]:
@step(
    name="ModelTraining",
    instance_type=instance_type,
)
def model_training(train_path:str, test_path:str,periodo:int)->str:
    print(f"Train set: {train_path} and test set:{test_path}")
    print(f"Periodo:{periodo}")
    return "model_path"

In [60]:
data_pull_step = data_pull("s3:mybucket/data",periodo)
model_training_step = model_training(data_pull_step[0],data_pull_step[1],periodo)

In [61]:
pipeline = Pipeline(name=pipeline_name, steps=[data_pull_step, model_training_step], parameters=[periodo])
pipeline.upsert(role_arn=role)

2025-06-01 00:58:53,486 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-635106763104/pipeline-prueba/DataPull/2025-06-01-00-58-53-238/function
2025-06-01 00:58:53,550 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-635106763104/pipeline-prueba/DataPull/2025-06-01-00-58-53-238/arguments
2025-06-01 00:58:53,773 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-635106763104/pipeline-prueba/ModelTraining/2025-06-01-00-58-53-238/function
2025-06-01 00:58:53,854 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-635106763104/pipeline-prueba/ModelTraining/2025-06-01-00-58-53-238/arguments
2025-06-01 00:58:54,280 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-635106763104/pipeline-prueba/DataPull/2025-06-01-00-58-54-280/function
2025-06-01 00

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:635106763104:pipeline/pipeline-prueba',
 'ResponseMetadata': {'RequestId': '4a1eb3d1-488c-4cc5-80d7-7f045830d645',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4a1eb3d1-488c-4cc5-80d7-7f045830d645',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '83',
   'date': 'Sun, 01 Jun 2025 00:58:55 GMT'},
  'RetryAttempts': 0}}

In [62]:
pipeline.start(parameters={"PeriodoCarga":202505})

_PipelineExecution(arn='arn:aws:sagemaker:us-east-1:635106763104:pipeline/pipeline-prueba/execution/lcytyhv4x0gi', sagemaker_session=<sagemaker.session.Session object at 0x7f2f2947f200>)

In [ ]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()
execution.describe()
execution.wait()
execution.list_steps()

# PIPELINE

